In [13]:
from pymcdm.methods import TOPSIS
import pandas as pd
import numpy as np
from mealpy.swarm_based.PSO import OriginalPSO
from pymcdm_reidentify.methods import STFN
from pymcdm_reidentify.normalizations import FuzzyNormalization

In [14]:
matrix = np.random.random((10, 5))
types = np.array([1, 1, 1, -1, -1])
weights = np.array([0.15, 0.228, 0.222, 0.21, 0.19]) # np.array([0.25, 0.25, 0.25, 0.25])

In [15]:
topsis = TOPSIS()
expert_pref = topsis(matrix, weights, types)
print("pref =", np.round(expert_pref, 4))
expert_rank = topsis.rank(expert_pref)
print("rank =", expert_rank)


pref = [0.5836 0.4071 0.5712 0.6515 0.5376 0.6009 0.4269 0.3187 0.5733 0.514 ]
rank = [ 3.  9.  5.  1.  6.  2.  8. 10.  4.  7.]


In [16]:
def make_bounds(data_matrix):
    bounds = np.array([[np.min(data_matrix[:, i]), np.max(data_matrix[:, i])]
                      for i in range(data_matrix.shape[1])])
    return bounds

In [17]:
bounds = make_bounds(matrix)
bounds

array([[0.12188893, 0.98154143],
       [0.13822832, 0.89279195],
       [0.09630168, 0.92696656],
       [0.03408167, 0.94839913],
       [0.0458324 , 0.81698017]])

In [18]:
stoch = OriginalPSO(epoch=1000, pop_size=100)
stfn = STFN(stoch.solve, TOPSIS(), bounds, weights)

In [19]:
stfn.fit(matrix, expert_rank)

2025/12/07 12:34:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2025/12/07 12:34:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 1, Current best: 0.7179063360881544, Global best: 0.7179063360881544, Runtime: 0.04346 seconds
2025/12/07 12:34:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 2, Current best: 0.7179063360881544, Global best: 0.7179063360881544, Runtime: 0.03131 seconds
2025/12/07 12:34:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 3, Current best: 0.7179063360881544, Global best: 0.7179063360881544, Runtime: 0.02879 seconds
2025/12/07 12:34:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 4, Current best: 0.7179063360881544, Global best: 0.7179063360881544, Runtime: 0.02789 seconds
2025/12/07 12:34:55 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >>>Problem: P, Epoch: 5, Current best: 0.7179063360881544, Global best: 0.717906336088154

In [20]:
print(f"STFN cores: {stfn.cores}")

STFN cores: [0.57115736 0.19498067 0.5095077  0.22845642 0.7589025 ]


In [21]:
def rw(rankx, ranky, n):
    suma = 0
    for i in range(n):
        suma += ((
            (rankx[i]-ranky[i])**2)
            *((n-rankx[i]+1)+(n-ranky[i]+1)
                    ))
    suma = 6 * suma
    denominator = n**4 + n**3 - n**2 - n
    if denominator == 0:
        return 0
    suma = suma / denominator
    return 1-suma

def WS(rankx, ranky, n):
    suma = 0
    for i in range(n):
        eq = 2 ** (-float(rankx[i]))
        eq2 = abs(rankx[i] - ranky[i]) / max(abs(1 - rankx[i]), abs(n - rankx[i]))
        suma += eq * eq2
    return 1 - suma

In [22]:
ob_norm = FuzzyNormalization(stfn())
body = TOPSIS(ob_norm)
new_types = np.ones(matrix.shape[1])
new_pref = body(matrix, weights, new_types)
new_rank = body.rank(new_pref)

In [23]:
print("new rank:", new_rank)
print("old rank:", expert_rank)
print("RW:", rw(expert_rank, new_rank, matrix.shape[0]))
print("WS:", WS(expert_rank, new_rank, matrix.shape[0]))

new rank: [ 3.  7.  4.  1. 10.  2.  6.  9.  8.  5.]
old rank: [ 3.  9.  5.  1.  6.  2.  8. 10.  4.  7.]
RW: 0.7972451790633609
WS: 0.9352663070436508


In [24]:
matrix

array([[0.91845737, 0.28924424, 0.48644116, 0.03408167, 0.23422334],
       [0.31096791, 0.16280384, 0.21537207, 0.14200495, 0.35449645],
       [0.98154143, 0.79854272, 0.27983775, 0.24277875, 0.61476213],
       [0.47756564, 0.89279195, 0.47958452, 0.12500133, 0.4364512 ],
       [0.88444008, 0.87396469, 0.19314918, 0.8856396 , 0.0458324 ],
       [0.87393017, 0.52632032, 0.69478737, 0.55479521, 0.3172138 ],
       [0.43521832, 0.55529688, 0.49081258, 0.94839913, 0.31525318],
       [0.12188893, 0.13822832, 0.09630168, 0.13291903, 0.81698017],
       [0.50313424, 0.48615464, 0.92696656, 0.69613057, 0.2629358 ],
       [0.97829478, 0.33298489, 0.75882595, 0.65228679, 0.47074824]])